# Inception

This work presents implemeting Inception model using Keras

<h3 style='color:red'>Documentation </h3>
<pre>
**class Model: Has all methods for geting data, perpare it, build
               the structure of NN, train and evaluate it.**
*Methods*:
               


- get_data          : This function for Loading cifar data - 
                      images and labels.
- normalize_get_data: This function for normalizing the data and 
                      split it.
- inception_block   : This function for building the inception block.  
- model             : This function for building the whole model.
- plot              : This function for plotting figures, descriping 
                      the realtion between accuracy and epochs.
- save_info         : This function to save info. about the model.
</pre>

In [14]:
import keras 
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer

In [18]:
class Model():
    def __init__(self):
        self.train_model()
    
    
    def get_data(self):
        '''
        This function for reading the data from the file and store it
        in numpy array as images and labels
        
        returns:-
        images: numpy array of size (50000, 32, 32, 3).
        lable : numpy array of size (50000, 1). 
        
        '''
        
        # For storeing the output of the function
        images = []
        labels = []
        
        # Loop through each file in the folder to be read.
        for batch_id in range(1, 6):
            
            # Open the file and read the data.
            with open('cifar-10-batches-py' + '/data_batch_' + str(batch_id), mode='rb') as file:
                # note the encoding type is 'latin1'
                batch = pickle.load(file, encoding='latin1')
            
            # append each patch to other patches
            images.append(batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1).astype("uint8"))
            labels.append(batch['labels'])
        
        # Now the images array has the size (5, 10000, 32, 32, 3) let's reshape it 
        # to the shape (50000, 32, 32, 3). Also reshaping labels array to have the 
        # size (50000,1)
        images = np.reshape(np.array(images), (50000, 32, 32, 3))        
        labels = np.reshape(np.array(labels), (50000, -1))
        
        # return the result.
        return images, labels
    
    
    def normalize_get_data(self):
        '''
        This function for normalize input between [-1, 1] 
        using mean and std. 
        parmas: no.
        returns: 
                train_x : numpy array of size (5000, 32, 32, 3).
                val_x   : numpy array of size (1000, 32, 32, 3).
                train_y : numpy array of size (5000, 1). 
                val_y   : numpy array of size (1000, 1).
        
        '''
        # Get batches
        features, labels = self.get_data()
        
        # Normalize data
        mean = features.mean()
        std = features.std()
        features = (features - mean)/std
        
        '''
        Split the data into training set and testing set.
        In this code training set is 5000 and validation set =1000.
        NOTE:
        YOU CAN CHANGE THE SIZE OF DATA.
        '''
        spliter = 5000
        train_x, val_x = features[0:spliter], features[spliter:6000]
        train_y, val_y = labels[0:spliter], labels[spliter:6000]
    
        
        return {"data":(train_x, train_y, val_x, val_y), "statistic":(mean, std)}
    
    
    def inception_block(self, x):
        '''
        This function for building just inception block and 
        calculate output of it as descriped in readme. 
        params:
            x: features.
        returns:
            output of inception block. 
        '''
        
        a_1 = keras.layers.Conv2D(64, (1,1), strides=1, padding='same', data_format='channels_last', )(x)
        a_1 = keras.layers.BatchNormalization(axis=3, momentum=.9)(a_1)
        a_1 = keras.layers.Activation("relu")(a_1)

        a_2 = keras.layers.Conv2D(96, (1,1), strides=1, padding='same', data_format='channels_last')(x)
        a_2 = keras.layers.BatchNormalization(axis=3, momentum=.9)(a_2)
        a_2 = keras.layers.Activation("relu")(a_2)
        a_2 = keras.layers.Conv2D(128, (3,3), strides=1, padding='same', data_format='channels_last')(a_2)
        a_2 = keras.layers.BatchNormalization(axis=3, momentum=.9)(a_2)
        a_2 = keras.layers.Activation("relu")(a_2)

        a_3 = keras.layers.Conv2D(16, (1,1), strides=1, padding='same', data_format='channels_last')(x)
        a_3 = keras.layers.BatchNormalization(axis=3, momentum=.9)(a_3)
        a_3 = keras.layers.Activation("relu")(a_3)
        a_3 = keras.layers.Conv2D(32, (5,5), strides=1, padding='same', data_format='channels_last')(a_3)
        a_3 = keras.layers.BatchNormalization(axis=3, momentum=.9)(a_3)
        a_3 = keras.layers.Activation("relu")(a_3)

        a_4 = keras.layers.MaxPooling2D((1, 1), padding='same')(x)
        a_4 = keras.layers.Conv2D(32, (1,1), strides=1, padding='same', data_format='channels_last')(a_4)
        a_4 = keras.layers.Activation("relu")(a_4)
        
        # Concancate all the layers.
        out = keras.layers.Concatenate(axis=-1)([a_1, a_2, a_3, a_4])
        return out

    def max_pool(self, x):
        
        '''
        This function for computing maxPooling that shrinks the 
        size by two.
        params:
            x: input from previous layer.
        returns:
            the out from maxPlooling.
        
        '''
        x = keras.layers.MaxPooling2D((2, 2))(x)
        return x 
    
    
    def model(self, input_shape):
        '''
        This function for building the whole model as persented
        in the paper except that I used just one block of 
        inception model.
        
        params:
            input_shape: Tuple of the input shape. 
        returns:
            model: the instamce of the model
        '''
        
        input_x = keras.layers.Input(shape=input_shape)
        x = keras.layers.Conv2D(32, (3, 3), strides = 1, padding='same', data_format = "channels_last", activation="relu")(input_x)
        x = self.max_pool(x)
        x = keras.layers.BatchNormalization(momentum = .9)(x)
        
        x = self.inception_block(input_x)
        x = self.max_pool(x)

        x = keras.layers.Conv2D(16, (1,1), strides=1, padding='same', data_format='channels_last', activation="relu")(x)

        x = keras.layers.AveragePooling2D((3,3), strides=2)(x)
        
        x= keras.layers.Flatten()(x)

        x = keras.layers.Dropout(rate = 1 - .5)(x)

        x = keras.layers.Dense(64, activation="relu")(x)

        x = keras.layers.Dense(10, activation="softmax")(x)

        model = keras.models.Model(input_x, x)

        return model

   
   
    def plot(self, history, file_name):
        
        '''
        This function for ploting acc vs epochs 
        and loss vs epochs.
        
        input: 
            history: Dict containing information about 
                     acc and loss.
            file_name: The perfix name of current model - 
                       type: string. 
        '''
        # Plot training & validation accuracy values
        file_name_acc = file_name + "_acc.png"
        plt.plot(history.history['acc'])
        plt.plot(history.history['val_acc'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Val'], loc='upper left')
        plt.savefig(file_name_acc)
        plt.close()
        
        # Plot training & validation loss values
        file_name_loss = file_name + "_loss.png"
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Val'], loc='upper left')
        plt.savefig(file_name_loss)
        plt.close()
        
    def save_info(self, history, lr, beta, batch_size, file_name):
        '''
        This fuction for saving information of the model. The info are
        accuracies, learning rate, beta, batch size.
        params:
            history: Dict containing all information  about current model.
            lr     : Learning rate - type: float.
            beta:  hyperparameters used in Adam, Adagrad, ...... -type: float.
            batch_size: Number of input in each epoch type- float.
            file_name: Perfix of the file name - type: string.
        '''
    
        info = " lr = {} \n beta = {} \n batch_size = {} \n acc = {} \n val_acc = {} \n loss = {} \n val_loss = {}".format(
        lr, beta, batch_size, history.history['acc'][-1], history.history['val_acc'][-1],\
            history.history['loss'][-1], history.history['val_loss'][-1])

        file_name += ".txt"
        file = open(file_name, 'w')
        file.write(info)
        file.close()
    
    def train_model(self):
        # Get data
        raw_data = self.normalize_get_data()
        train_x, train_y, val_x, val_y = raw_data["data"]
        
        # Loop over optimizers
        for batch_size in [64, 128, 256]:
            for lr in [.01, .001, .0001]:
                for beta in [.9, .99, .999]:
                    print("batch_size is {}, lr is {}, beta is {}".format(batch_size, lr, beta))
                    cifarModel = self.model(train_x[0].shape)
                    optimizer = keras.optimizers.Adam(lr=lr, beta_1=beta, beta_2=beta, epsilon=10e-8, decay=0.0, amsgrad=False)
                    cifarModel.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
                    file_name = "results/lr_"+ str(lr) + "_beta_" + str(beta) + "_batchsize_" + str(batch_size)
                    checkpointer = keras.callbacks.ModelCheckpoint(filepath=file_name +".hdf5", verbose=0, save_best_only=True)
                    history = cifarModel.fit(x=train_x, y=keras.utils.to_categorical(train_y), \
                                   batch_size=batch_size, epochs=50, validation_data=(val_x, keras.utils.to_categorical(val_y)),\
                                   callbacks=[checkpointer])
                    self.plot(history, file_name)
                    self.save_info(history, lr, beta, batch_size, file_name)
                    
                  
        

In [ ]:
obj = Model()

batch_size is 64, lr is 0.01, beta is 0.9
Train on 5000 samples, validate on 1000 samples
Epoch 1/50
 256/5000 [>.............................] - ETA: 18:23 - loss: 2.7748 - acc: 0.1016